# Implementing Portfolio Insurance (CPPI) and Drawdown Constraints

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import risk_kit as rk
import numpy as np
import pandas as pd

ind_return = rk.get_ind_returns()
tmi_return = rk.get_total_market_index_returns()

In [2]:
risky_r = ind_return["2000":][["Steel", "Fin", "Beer"]]
#Safe Asset
safe_r = pd.DataFrame().reindex_like(risky_r)

In [3]:
safe_r[:] = 0.03/12
start = 1000
floor = 0.8

In [4]:
def compound(r) :
    return (1+r).prod()-1

def compound2(r):
    return np.expm1(np.log1p(r).sum())

In [5]:
compound(risky_r)

Steel   -0.051696
Fin      1.773937
Beer     3.361349
dtype: float64

In [6]:
compound2(risky_r)

Steel   -0.051696
Fin      1.773937
Beer     3.361349
dtype: float64

In [7]:
%timeit compound(risky_r)

250 µs ± 884 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%timeit compound2(risky_r)

226 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
dates = risky_r.index
n_steps = len(dates)
account_value = start
floor_value = start*floor
m = 3
account_history = pd.DataFrame().reindex_like(risky_r)
cushion_history = pd.DataFrame().reindex_like(risky_r)
risky_w_history = pd.DataFrame().reindex_like(risky_r)

for step in range(n_steps):
    cushion = (account_value - floor_value)/account_value
    risky_w = m*cushion
    risky_w = np.minimum(risky_w, 1)
    risky_w = np.maximum(risky_w, 0)
    safe_w = 1-risky_w
    risky_alloc = account_value*risky_w
    safe_alloc = account_value*safe_w
    ## update the account value for this time step
    account_value = (risky_alloc*(risky_r.iloc[step]) + safe_alloc*safe_r.iloc[step]
    #save trhe values so I can look at the history and plot it etc
    cushion_history.iloc[step] = cushion
    risky_w_history.iloc[step] = risky_w
    account_history.iloc[step] = account_value

SyntaxError: invalid syntax (<ipython-input-10-6ef023295fc2>, line 21)

In [ ]:
risky_wealth = start*(1+risky_r).cumprod()

In [ ]:
risky_wealth.plot()

In [ ]:
ax = account_history["Beer"].plot(figsize=(12,6))
risky_wealth["Beer"].plot(ax=ax, style="k:")
ax.axhline(y=floor_value, color='r', linestyle='--')

In [ ]:
risky_w_history.plot()